 we use pre-trained model from google's tensorflow hub , and re-train it on new dataset, and we will add new layers to it for our new dataset

Using pre-trained model saves lot of time and computational budget for new classification problem at hand

google's model has 1.4 mil images and 1000 classes , we can use this for our image classification

we freeze all layers except the last one which is our activation layer , to use it on new dataset

when we freeze these layers , the model weights won't change

when we perform training on new dataset weights won't change

then we use softmax to classify it into n number of classes we want in our new dataset

# we will use # mobilenet v2 model here:

In [7]:
import numpy as np
import cv2

import PIL.Image as image
import os

import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
tf.__version__

'2.9.0'

In [2]:
IMAGE_SHAPE = (224,224)

classifier = keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4', input_shape= IMAGE_SHAPE+(3,)) # adding third dimension to image_shape
])

test the model if it works , whith new goldfish image

In [3]:
gold_fish = image.open('datasets//transfer_learning//goldfish.jpg').resize(IMAGE_SHAPE)

In [4]:
gold_fish = np.array(gold_fish)/255.0  # convert image into array and scale it

In [5]:
gold_fish[np.newaxis, ...].shape  # adding index dimension , classifier accept data with index

(1, 224, 224, 3)

In [6]:
result = classifier.predict(gold_fish[np.newaxis, ...])

1/1 [==============================] - 1s 563ms/step


In [7]:
result.argmax() # it is index in our classes

2

we can use classes txt file

In [8]:
image_labels = []
with open("datasets//transfer_learning//ImageNetLabels.txt", "r") as f:
    image_labels = f.read().splitlines() # it will split the line , each class written in one line
image_labels[:5]

['background', 'tench', 'goldfish', 'great white shark', 'tiger shark']

In [9]:
image_labels[result.argmax()] 

'goldfish'

# model predicted it correctly!!

loading new dataset(flower photos)

#### I used tensorflow offical tutorial: https://www.tensorflow.org/tutorials/images/classification as a reference and made bunch of changes to make it simpler

In [10]:
data_dir = ".//datasets//flower_photos//flower_photos"

In [11]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos/flower_photos')

In [12]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [13]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [14]:
x = []
y = []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image)) # it will load image as array
        resized_img = cv2.resize(img, (224,224))  # we will resize all images so all of them will be same size
        x.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [15]:
x = np.array(x)
y = np.array(y)

In [16]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(x, y, test_size=0.2, random_state=0)

In [21]:
x_train_scaled = x_train / 255
x_test_scaled = x_test / 255

In [26]:
predicted = classifier.predict(np.array([x[0], x[1], x[2]]))
predicted = np.argmax(predicted, axis=1)
predicted

1/1 [==============================] - 1s 850ms/step


array([795, 880, 795], dtype=int64)

it will predict it based on labels we had in pre-trained model

In [28]:
image_labels[880]

'umbrella'

# now let's modify the model for our new problem

In [29]:
feature_extractor_model = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4'
pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable= False  # trainable False means freeze the layers
)

In [30]:
num_classes = 5

model = tf.keras.Sequential([
    pretrained_model_without_top_layer,
    layers.Dense(num_classes)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1001)              3540265   
                                                                 
 dense (Dense)               (None, 5)                 5010      
                                                                 
Total params: 3,545,275
Trainable params: 5,010
Non-trainable params: 3,540,265
_________________________________________________________________


only last layer is our addition to the model and previous layers are from the pre-trained model

In [31]:
model.compile(
    optimizer='adam',
    loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc']
)

In [32]:
model.fit(x_train_scaled, y_train, epochs= 10)

Epoch 1/10
92/92 [==============================] - 63s 630ms/step - loss: 0.8075 - acc: 0.7132
Epoch 2/10
92/92 [==============================] - 58s 633ms/step - loss: 0.3860 - acc: 0.8593
Epoch 3/10
92/92 [==============================] - 55s 599ms/step - loss: 0.3040 - acc: 0.8924
Epoch 4/10
92/92 [==============================] - 56s 606ms/step - loss: 0.2550 - acc: 0.9111
Epoch 5/10
92/92 [==============================] - 56s 608ms/step - loss: 0.2217 - acc: 0.9285
Epoch 6/10
92/92 [==============================] - 57s 618ms/step - loss: 0.1989 - acc: 0.9332
Epoch 7/10
92/92 [==============================] - 55s 601ms/step - loss: 0.1766 - acc: 0.9438
Epoch 8/10
92/92 [==============================] - 56s 608ms/step - loss: 0.1632 - acc: 0.9482
Epoch 9/10
92/92 [==============================] - 55s 598ms/step - loss: 0.1503 - acc: 0.9547
Epoch 10/10
92/92 [==============================] - 55s 596ms/step - loss: 0.1355 - acc: 0.9622


# here we can see that we have much better accuracy based on pre-trained model

here with less epochs we reach higher accuracy , which means we have to do less computation on our machine and with less time spent we reach higher accuracy

# it is all because of transfer learning

this is why transfer learning is popular in computer vision

In [33]:
model.evaluate(x_test_scaled, y_test)

23/23 [==============================] - 16s 622ms/step - loss: 0.4614 - acc: 0.8597


[0.46137717366218567, 0.859673023223877]